## Lab-8

Name: Purusharth Malik

Registration No.: 2348542

### 1. Develop a Python script to load the fine-tuned model and tokenizer.

In [1]:
from transformers import AutoTokenizer, AutoModel
# Loading the tokenizer
tokenizer=AutoTokenizer.from_pretrained("ai-models/xlm-roberta-base-finetuned-imdb")
# Loading the fine-tuned model
model=AutoModel.from_pretrained("ai-models/xlm-roberta-base-finetuned-imdb")

tokenizer_config.json:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

Some weights of XLMRobertaModel were not initialized from the model checkpoint at ai-models/xlm-roberta-base-finetuned-imdb and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### 2. Develop a Python script for handling multimodal inputs.

In [2]:
import torch
from transformers import AutoTokenizer, AutoModel
from PIL import Image
from torchvision import transforms, models

# Loading the model
text_model_name = 'bert-base-uncased'
image_model = models.resnet50(pretrained=True)
tokenizer = AutoTokenizer.from_pretrained(text_model_name)
text_model = AutoModel.from_pretrained(text_model_name)

# Image processing transformations
image_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Preprocessing function for text
def process_text(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        outputs = text_model(**inputs)
    text_embedding = outputs.last_hidden_state.mean(dim=1) # Average over sequence
    return text_embedding

# Preprocessing function for image
def process_image(image_path):
    image = Image.open(image_path).convert('RGB')
    image_tensor = image_transforms(image).unsqueeze(0)
    with torch.no_grad():
        image_embedding = image_model(image_tensor)
    return image_embedding

# Concatinating the embeddings
def combine_embeddings(text_embedding, image_embedding):
    # Example combination; could use concatenation, addition, or another fusion method
    combined = torch.cat((text_embedding, image_embedding), dim=1)
    return combined

text = "I am human."
image_path = 'image.jpg'

text_emb = process_text(text)
image_emb = process_image(image_path)
combined_emb = combine_embeddings(text_emb, image_emb)

print("Text Embedding Shape:", text_emb.shape)
print("Image Embedding Shape:", image_emb.shape)
print("Combined Embedding Shape:", combined_emb.shape)

C:\Users\purus\Anaconda3\envs\dragon\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\purus\Anaconda3\envs\dragon\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\purus/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth
100%|█████████████████████████████████████████████████████████████████████████████| 97.8M/97.8M [00:08<00:00, 11.7MB/s]


Text Embedding Shape: torch.Size([1, 768])
Image Embedding Shape: torch.Size([1, 1000])
Combined Embedding Shape: torch.Size([1, 1768])


# End